<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Biomass Modeling

_by Jeremy Freeman (CarbonPlan), September 19, 2020_

This notebook extracts fits predictive biomass growth curves from FIA data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from forests import load, setup, plot, fit

In [3]:
setup.plotting()

In [54]:
df = load.fia(store='local', states='conus')
df = load.terraclim(store='local', tlim=(2000,2020), mean=True, df=df)

In [12]:
type_codes = df['type_code'].unique()

In [ ]:
foo = load.fia(store='local', states='NY', wide=True)

In [69]:
# fit all models
models = {}
for i, code in enumerate(type_codes):
    print(f'code: {code}')
    print(f'percent complete: {100 * i/len(type_codes)}%')
    df_type = df[df['type_code'] == code].reset_index()
    if len(df_type) > 30:
        x = df_type['age']
        y = df_type['biomass']
        f = [df_type['tmax'], df_type['ppt']]
        model = fit.biomass(x=x, y=y, f=f, noise='gamma')
        models[code] = model
    else:
        print('skipping due to low n')

code: 407.0
percent complete: 0.0%
code: 141.0
percent complete: 0.6172839506172839%
code: 403.0
percent complete: 1.2345679012345678%
code: 161.0
percent complete: 1.8518518518518519%
code: 406.0
percent complete: 2.4691358024691357%
code: 508.0
percent complete: 3.0864197530864197%
code: 607.0
percent complete: 3.7037037037037037%
code: 409.0
percent complete: 4.320987654320987%
code: 520.0
percent complete: 4.938271604938271%
code: 608.0
percent complete: 5.555555555555555%
code: 503.0
percent complete: 6.172839506172839%
code: 706.0
percent complete: 6.790123456790123%
code: 602.0
percent complete: 7.407407407407407%
code: 506.0
percent complete: 8.024691358024691%
code: 171.0
percent complete: 8.641975308641975%
code: 510.0
percent complete: 9.25925925925926%
code: 142.0
percent complete: 9.876543209876543%
code: 501.0
percent complete: 10.493827160493828%
code: 405.0
percent complete: 11.11111111111111%
code: 404.0
percent complete: 11.728395061728396%
code: 502.0
percent complet

In [ ]:
code

In [ ]:
# evaluate predictions across target years

code = 221
inds = df['type_code'] == code

mf = pd.DataFrame()
mf['lat'] = df[inds]['lat']
mf['lon'] = df[inds]['lon']

targets = np.arange(2000,2100,5)
for it in range(len(targets)):
    x = df[inds]['age']
    year = df[inds]['year']
    f = [df[inds]['tmax'], df[inds]['ppt']]
    if it == 0:
        mf[targets[it]] = models[code].predict(np.maximum(x + (targets[it] - year), 0), f)
    else:
        diff = models[code].predict(x + (targets[it] - year), f) - models[code].predict(x + (targets[it-1] - year), f)
        mf[targets[it]] = np.maximum(mf[targets[it-1]] + diff, 0)

In [ ]:
plt.plot?

In [ ]:
from palettable.cubehelix import perceptual_rainbow_16
cmap = perceptual_rainbow_16.mpl_colormap

In [ ]:
import matplotlib as mpl

mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

In [ ]:
plt.figure(figsize=[12,5])
pf = mf[targets].sort_values(2000).reset_index(drop=True)
for row,value  in pf.iterrows():
    plt.plot(value, 'b', color=cmap(row/len(pf)), alpha=0.1);

In [ ]:
plt.plot(mf[targets].T, 'b', alpha=0.01);

In [ ]:
plt.hist([models[k].train_r2 for k in models.keys()])

In [ ]:
code

In [ ]:
df.groupby('type_code').agg(count=('type_code', 'count')).sort_values('count', ascending=False).T

In [60]:
inds = df['type_code'] == 983 # 261 is a problem, 941 / 371 is a good example, 341 shows model deviation
x = df[inds]['age']
y = df[inds]['biomass']
f = [df[inds]['tmax'], df[inds]['ppt']]
(
plot.xy(x=x, y=y, color=f[0], xlim=[0,200], ylim=[0,500])
|
plot.xy(x=x, y=y, color=f[1], xlim=[0,200], ylim=[0,500])
).resolve_scale(color='independent')

alt.HConcatChart(...)

In [ ]:
604.4287614874837

In [66]:
model = fit.biomass(x=x, y=y, f=f, noise='gamma')
print(model.result)

optimization failed
 barrier_parameter: 0.1
 barrier_tolerance: 0.1
          cg_niter: 29829
      cg_stop_cond: 1
            constr: [array([9.70052286e+01, 3.58466984e-02, 3.57805127e+00, 1.44647089e+00,
       4.65208110e-02, 6.45373609e+01])]
       constr_nfev: [0]
       constr_nhev: [0]
       constr_njev: [0]
    constr_penalty: 10.980954230232987
  constr_violation: 0.0
    execution_time: 18.522501945495605
               fun: 612.558647448375
              grad: array([-0.00387969,  1.26554108, -0.00706636, -0.10988342, -2.37420654,
        0.35317159])
               jac: [<6x6 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>]
   lagrangian_grad: array([-4.91004250e-03, -1.95586287e-03, -3.98583260e-02, -1.09883425e-01,
       -2.37420654e+00,  3.51537703e-01])
           message: 'The maximum number of function evaluations is exceeded.'
            method: 'tr_interior_point'
              nfev: 686
              nh

In [67]:
model.r2(x, f, y)

0.386607329178414

In [68]:
xlim = [0, 150]
ylim= [0, 1000]
(
(
plot.xy(x=x, y=y, color=f[0], xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [90, 50]), color=np.nanpercentile(f[0], 90), xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [10, 50]), color=np.nanpercentile(f[0], 10), xlim=xlim, ylim=ylim)
)
|
(
plot.xy(x=x, y=y, color=f[1], xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [50, 10]), color=np.nanpercentile(f[1], 10), xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [50, 90]), color=np.nanpercentile(f[1], 90), xlim=xlim, ylim=ylim)
)
).resolve_scale(color='independent')

alt.HConcatChart(...)

In [ ]:
xlim = [0,150]
ylim= [-100,400]
((plot.xy(x=x, y=y, xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [50, 50]), xlim=xlim, ylim=ylim))
|
(plot.xy(x=x, y=model.sample(x, f), xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [50, 50]), xlim=xlim, ylim=ylim)))

In [ ]:
plot.carto(lat=df['lat'], lon=df['lon'], color=df['biomass'], cmap='yellowgreen', clim=(0,200))

In [ ]:
plot.carto(lat=df['lat'], lon=df['lon'], color=df['tmax'], cmap='plasma', clim=(5, 30))

In [ ]:
import rasterio

In [ ]:
r = rasterio.open('/Users/freeman/workdir/carbonplan-data/processed/nlcd/conus/4000m/2016_c42.tif')

In [ ]:
a = r.read(1)

In [ ]:
from showit import image
%matplotlib inline

In [ ]:
a.shape

In [ ]:
image(a, size=12)

In [ ]:
from rasterio import Affine
from rasterio.transform import rowcol
from pyproj import transform, Proj
from carbonplan_data.utils import albers_conus_crs, albers_conus_transform

In [ ]:
inds = df['year'] > 2010
df_sub = df[inds].reset_index()
t = Affine(*albers_conus_transform(4000))
p1 = Proj(albers_conus_crs())
p2 = Proj(proj='latlong', datum='WGS84')
x, y = transform(p2, p1, df_sub['lon'].values, df_sub['lat'].values)
rc = rowcol(t, x, y)

In [ ]:
m = np.zeros(a.shape)
b = np.zeros(a.shape)

In [ ]:
for i, (r, c) in enumerate(zip(rc[0],rc[1])):
    m[r, c] += 1
    b[r, c] += df_sub['biomass'][i]

In [ ]:
m[rc[0], rc[1]] = df['biomass']

In [ ]:
image(a[0:300,0:300], size=12)

In [ ]:
image(b[0:300,0:300] / m[0:300,0:300], clim=(0,400), size=12)

In [ ]:
image(m[0:300,0:300], clim=(0,1), size=12)

In [ ]:
image(m, clim=(0,200), cmap='viridis', size=12)

In [ ]:
image((m[0:400,0:400]!=201) & (m[0:400,0:400]!=0), size=12)

In [ ]:
np.unique(a[0:400,0:400])

In [ ]:
image(((m[0:400,0:400]!=201) & (m[0:400,0:400]!=0)) + 0.5*(a[0:400,0:400]==301), size=12)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sum(types==df['type_code'])/len(types)

In [ ]:
plot.xy(x=types, y=df['type_code'])

In [ ]:
image(m[0:300,0:300], size=12)